In [5]:
import openai
import tomli
import requests
import json

In [ ]:
with open('.streamlit/secrets.toml','rb') as f:
    toml_dict = tomli.load(f)

In [ ]:
openai.api_key = toml_dict['OPEN_AI_KEY']

In [13]:
url = "https://www.laforet.com/agence-immobiliere/meudon/acheter"
# url = "https://www.orpi.com/recherche/buy?transaction=buy&resultUrl=&realEstateTypes%5B0%5D=maison&realEstateTypes%5B1%5D=appartement&locations%5B0%5D%5Bvalue%5D=meudon&locations%5B0%5D%5Blabel%5D=Meudon%20%2892190%29&agency=agenceorangerie&minSurface=&maxSurface=&minLotSurface=&maxLotSurface=&minStoryLocation=&maxStoryLocation=&newBuild=&oldBuild=&minPrice=&maxPrice=&sort=date-down&layoutType=mixte&page=&recentlySold=false"

In [14]:
html = requests.get(url)
html.text

'<!doctype html>\n<html data-n-head-ssr lang="fr" data-n-head="%7B%22lang%22:%7B%22ssr%22:%22fr%22%7D%7D">\n  <head >\n    <!--\n\'##::::::::::\'###::::\'########::\'#######::\'########::\'########:\'########:\n ##:::::::::\'## ##::: ##.....::\'##.... ##: ##.... ##: ##.....::... ##..::\n ##::::::::\'##:. ##:: ##::::::: ##:::: ##: ##:::: ##: ##:::::::::: ##::::\n ##:::::::\'##:::. ##: ######::: ##:::: ##: ########:: ######:::::: ##::::\n ##::::::: #########: ##...:::: ##:::: ##: ##.. ##::: ##...::::::: ##::::\n ##::::::: ##.... ##: ##::::::: ##:::: ##: ##::. ##:: ##:::::::::: ##::::\n ########: ##:::: ##: ##:::::::. #######:: ##:::. ##: ########:::: ##::::\n........::..:::::..::..:::::::::.......:::..:::::..::........:::::..:::::\n    -->\n    <script>localStorage.getItem("browserId")||localStorage.setItem("browserId",Math.random().toString(36).substring(2,15)+Math.random().toString(36).substring(2,15)),dataLayer=[{browserId:localStorage.getItem("browserId"),contact:localStorage.getItem

In [12]:
with open('page.html','w') as f:
    f.write(html.text)

In [32]:
from bs4 import BeautifulSoup
# Parse the content using BeautifulSoup
soup = BeautifulSoup(html.content, 'html.parser')

# Search for the article elements
prop = soup.find_all(class_='card-bottom')
print(len(prop))
prop[0].text

8


'\n        Appartement\n       \n        515\u202f000\xa0€\n       Meudon \n        70 m²\n      \n        3 pièces\n      \n        2 chambres\n         '

In [37]:
text = [p.text for p in prop]
''.join(text)

'\n        Appartement\n       \n        515\u202f000\xa0€\n       Meudon \n        70 m²\n      \n        3 pièces\n      \n        2 chambres\n         \n        Appartement\n       \n        263\u202f000\xa0€\n       Palaiseau \n        69 m²\n      \n        3 pièces\n      \n        2 chambres\n         \n        Maison\n       \n        875\u202f000\xa0€\n       Clamart \n        130 m²\n      \n        5 pièces\n      \n        3 chambres\n         \n        Appartement\n       \n        395\u202f000\xa0€\n       Meudon \n        69 m²\n      \n        4 pièces\n      \n        3 chambres\n         \n        Appartement\n       \n        347\u202f000\xa0€\n       Meudon \n        57 m²\n      \n        3 pièces\n      \n        2 chambres\n         \n        Appartement\n       \n        396\u202f000\xa0€\n       Meudon \n        52 m²\n      \n        3 pièces\n      \n        2 chambres\n         \n        Maison\n       \n        1\u202f560\u202f000\xa0€\n       Meudon \n    

In [ ]:
link = [a['href'] for a in prop]

In [39]:
prompt = 'Convert the given HTML to JSON with the schema {"ville": "string", "pièces": "number", "chambres": "number", "m²": "number", "prix": "number"}'
completion = openai.ChatCompletion.create(
      model='gpt-3.5-turbo',
      temperature=0,
      messages=[
        {'role': 'system', 'content': prompt },
        {'role': 'user', 'content': ''.join(text) }
            ]
)
data = json.loads(completion.choices[0].message.content)
data

[{'ville': 'Meudon', 'pièces': 3, 'chambres': 2, 'm²': 70, 'prix': 515000},
 {'ville': 'Palaiseau', 'pièces': 3, 'chambres': 2, 'm²': 69, 'prix': 263000},
 {'ville': 'Clamart', 'pièces': 5, 'chambres': 3, 'm²': 130, 'prix': 875000},
 {'ville': 'Meudon', 'pièces': 4, 'chambres': 3, 'm²': 69, 'prix': 395000},
 {'ville': 'Meudon', 'pièces': 3, 'chambres': 2, 'm²': 57, 'prix': 347000},
 {'ville': 'Meudon', 'pièces': 3, 'chambres': 2, 'm²': 52, 'prix': 396000},
 {'ville': 'Meudon', 'pièces': 6, 'chambres': 4, 'm²': 140, 'prix': 1560000},
 {'ville': 'Meudon', 'pièces': 5, 'chambres': 3, 'm²': 105, 'prix': 760000}]